In [42]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [43]:
pd.options.display.max_columns = None

In [44]:
custom_na_values = ["Sin información", "Sin informacion", 'Na', "Nd", 'No aplica']

In [45]:
def try_parse_integer(code_mun):
    try:
        code_mun = int(code_mun)
        if code_mun==0:
            return np.nan
        else:
            return code_mun
    except Exception:
        return np.nan

In [46]:
def round_year(year, interval):
    return (year // interval) * interval

In [47]:
GROUP_YEARS_BY = 10

## Desaparición

In [48]:
df_desaparicion = pd.read_csv('../../data/memoria_historica/victimas_desaparicion.csv', na_values=custom_na_values)

In [23]:
round_year(df_desaparicion["ANNOH"], 10)

0        1990
1        2000
2        2000
3        2000
4        2000
         ... 
80467    2000
80468    2000
80469    2000
80470    2000
80471    2000
Name: ANNOH, Length: 80472, dtype: Int64

In [49]:
df_desaparicion["ANNOH"] = df_desaparicion.ANNOH.replace({0: np.nan}).astype("Int64")
df_desaparicion["COD_DANE_MUN"] = df_desaparicion.COD_DANE_MUN.apply(try_parse_integer).astype("Int64")

In [50]:
df_desaparicion_mun = df_desaparicion.groupby([
    "COD_DANE_MUN", 
    round_year(df_desaparicion["ANNOH"], GROUP_YEARS_BY)
    ]) \
    .agg({"IdCaso": "count"}).reset_index().rename({"IdCaso": "Casos"}, axis="columns")
df_desaparicion_mun["Tipo_Evento"] = "Desaparicion"
df_desaparicion_mun

,COD_DANE_MUN,ANNOH,Casos,Tipo_Evento
0,5000,1960,8,Desaparicion
1,5000,1980,5,Desaparicion
2,5000,1990,31,Desaparicion
3,5000,2000,8,Desaparicion
4,5001,1960,12,Desaparicion
...,...,...,...,...
3050,99624,2000,4,Desaparicion
3051,99773,1980,20,Desaparicion
3052,99773,1990,37,Desaparicion
3053,99773,2000,136,Desaparicion


## MAP - MUSE

In [51]:
df_map_casos = pd.read_csv('../../data/memoria_historica/casos_map_muse.csv', na_values=custom_na_values)
df_map_casos["ANNOH"] = df_map_casos.ANNOH.replace({0: np.nan}).astype("Int64")

In [52]:
df_map = pd.read_csv('../../data/memoria_historica/victimas_map_muse.csv', na_values=custom_na_values)

In [53]:
df_map_casos = df_map_casos.merge(df_map.groupby("IdCaso").agg({"COD_DANE_MUN": "max"}).reset_index(), how="left", on="IdCaso")
df_map_casos["COD_DANE_MUN"] = df_map_casos["COD_DANE_MUN"].apply(try_parse_integer).astype("Int64")

In [54]:
df_map_mun = df_map_casos.groupby([
    "Modalidad", 
    "COD_DANE_MUN",
    round_year(df_map_casos["ANNOH"], GROUP_YEARS_BY)
]).agg({"IdCaso": "count"}).reset_index().rename({"IdCaso": "Casos", "Modalidad":"Tipo_Evento"}, axis="columns")
df_map_mun["Tipo_Evento"] = df_map_mun["Tipo_Evento"].replace({'Afectación': "Munición sin Explotar", 'Atentado': 'Explosión mina antipersonal'})
df_map_mun.drop(df_map_mun[df_map_mun["Tipo_Evento"]=="Incidente"].index, inplace=True)
df_map_mun

,Tipo_Evento,COD_DANE_MUN,ANNOH,Casos
0,Munición sin Explotar,5001,1990,2
1,Munición sin Explotar,5001,2000,2
2,Munición sin Explotar,5042,2000,1
3,Munición sin Explotar,5045,2000,2
4,Munición sin Explotar,5055,2000,1
...,...,...,...,...
735,Explosión mina antipersonal,95200,2010,1
736,Explosión mina antipersonal,97001,2000,10
737,Explosión mina antipersonal,97161,2000,5
738,Explosión mina antipersonal,97161,2010,2


## Secuestro

In [87]:
df_secuestro = pd.read_csv('../../data/memoria_historica/victimas_secuestro.csv', na_values=custom_na_values)
df_secuestro["COD_DANE_MUN"] = df_secuestro.COD_DANE_MUN.apply(try_parse_integer).astype("Int64")

In [88]:
df_secuestro

,IdCaso,Id,DIAH,MESH,ANNOH,COD_DANE_MUN,DEPTO_CASO,MUNINICIO_CASO,Sexo,Etnia,OcupacionVictima,CalidadVictima,TipoPoblacionVulnerable,MilitantePolitico,SituacionActualVictima,Circustancia_Muerte_en_Cautiverio,Tipo_Liberación,Días_Cautiverio,No_Veces_secuestrado,Grupo,DESCRIPCION_GRUPO,Estado
0,490161,24104,17,4,2000,13688,Bolívar,Santa rosa del sur,M,NaN,Empleado,Civil,NaN,NaN,Muerto en cautiverio,Muerto en medio de acción bélica,NaN,NaN,1.0,NaN,NaN,Publicado
1,79491,48549,15,11,2008,11001,"Bogotá, d.c.","Bogotá, d.c.",H,NaN,Funcionario público,Civil,Edil,Liberal,Muerto en cautiverio,Asesinato selectivo,NaN,30.0,1.0,NaN,NaN,Publicado
2,104504,48620,19,6,1975,44001,La guajira,Riohacha,H,NaN,Funcionario público,Civil,Concejal,Otro,Muerto en cautiverio,Asesinato selectivo,NaN,NaN,NaN,NaN,NaN,Publicado
3,331829,52650,19,9,1997,5142,Antioquia,Caracolí,H,NaN,NaN,Civil,NaN,NaN,Liberado,NaN,Otro,NaN,1.0,NaN,NaN,Publicado
4,128384,55994,9,1,2000,68615,Santander,Rionegro,H,NaN,Trabajo sin especificar,Civil,NaN,NaN,Muerto en cautiverio,Asesinato selectivo,NaN,10.0,1.0,NaN,NaN,Publicado
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37160,532496,580386,5,10,2010,5858,Antioquia,Vegachí,H,NaN,Estudiante,Civil,NaN,NaN,Liberado,NaN,Otro,1.0,1.0,NaN,NaN,Publicado
37161,532496,580387,5,10,2010,5858,Antioquia,Vegachí,H,NaN,Estudiante,Civil,NaN,NaN,Liberado,NaN,Otro,1.0,1.0,NaN,NaN,Publicado
37162,532496,580388,5,10,2010,5858,Antioquia,Vegachí,H,NaN,Estudiante,Civil,NaN,NaN,Liberado,NaN,Otro,1.0,1.0,NaN,NaN,Publicado
37163,532496,580389,5,10,2010,5858,Antioquia,Vegachí,H,NaN,Estudiante,Civil,NaN,NaN,Liberado,NaN,Otro,1.0,1.0,NaN,NaN,Publicado


In [89]:
df_secuestro_casos = pd.read_csv('../../data/memoria_historica/casos_secuestro.csv', na_values=custom_na_values)
df_secuestro_casos["ANNOH"] = df_secuestro_casos.ANNOH.replace({0: np.nan}).astype("Int64")

In [90]:
df_secuestro = df_secuestro.merge(df_secuestro_casos[["IdCaso", "Modalidad", "Modalidad_de_Secuestro", "Tipo_Secuestro", "Finalidad_del_Secuestro"]], how="left", on="IdCaso")

In [91]:
df_secuestro["CalidadVictima"].value_counts(dropna=False)

Civil          35790
Combatiente     1371
NaN                4
Name: CalidadVictima, dtype: int64

In [92]:
df_secuestro_mun = df_secuestro.groupby([
    "CalidadVictima", 
    "COD_DANE_MUN",
    round_year(df_secuestro["ANNOH"], GROUP_YEARS_BY)
]).agg({"IdCaso": "count"}).reset_index().rename({"IdCaso": "Casos", "CalidadVictima":"Tipo_Evento"}, axis="columns")
df_secuestro_mun['Tipo_Evento'] = 'Secuestro a ' + df_secuestro_mun['Tipo_Evento']
df_secuestro_mun

,Tipo_Evento,COD_DANE_MUN,ANNOH,Casos
0,Secuestro a Civil,5000,0,1
1,Secuestro a Civil,5000,1970,5
2,Secuestro a Civil,5000,1980,2
3,Secuestro a Civil,5000,1990,3
4,Secuestro a Civil,5000,2000,13
...,...,...,...,...
3213,Secuestro a Combatiente,95200,1990,192
3214,Secuestro a Combatiente,95200,2000,1
3215,Secuestro a Combatiente,95200,2010,1
3216,Secuestro a Combatiente,97001,1990,61


## Exportar

In [93]:
df_mun = pd.concat([df_desaparicion_mun, df_map_mun, df_secuestro_mun]) \
    .pivot_table("Casos", ["COD_DANE_MUN", "ANNOH"], "Tipo_Evento", fill_value=0)
df_mun

Tipo_Evento         Desaparicion  Explosión mina antipersonal  \
COD_DANE_MUN ANNOH                                              
5000         0                 0                            0   
             1960              8                            0   
             1970              0                            0   
             1980              5                            0   
             1990             31                            0   
...                          ...                          ...   
99773        0                 0                            0   
             1980             20                            0   
             1990             37                            0   
             2000            136                            0   
             2010             16                            1   

Tipo_Evento         Munición sin Explotar  Secuestro a Civil  \
COD_DANE_MUN ANNOH                                             
5000         0                          0                  1   
             1960                       0                  0   
             1970                       0                  5   
             1980                       0                  2   
             1990                       0                  3   
...                                   ...                ...   
99773        0                          0                  1   
             1980                       0                  3   
             1990                       0                  8   
             2000                       0                 42   
             2010                       0                 26   

Tipo_Evento         Secuestro a Combatiente  
COD_DANE_MUN ANNOH                           
5000         0                            0  
             1960                         0  
             1970                         0  
             1980                         0  
             1990                         0  
...                                     ...  
99773        0                            0  
             1980                         0  
             1990                         6  
             2000                         0  
             2010                         0  

[3785 rows x 5 columns]

In [94]:
df_mun.to_csv('Consolidado Municipio.csv')